In [699]:
import pandas as pd
import numpy as np
from urllib import request
import json

In [948]:
df = pd.read_csv('DaVinciBrain/Talendig_Clean.csv')

In [949]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   No.                             160 non-null    int64  
 1   Entrenamiento                   160 non-null    object 
 2   Email                           160 non-null    object 
 3   Egresado                        160 non-null    object 
 4   Matricula                       160 non-null    int64  
 5   Fecha de Nacimiento             19 non-null     object 
 6   Edad                            19 non-null     float64
 7   Precios                         159 non-null    object 
 8   Ubicacion                       148 non-null    object 
 9   Incluye pasantia                159 non-null    object 
 10  Metodo de pago                  159 non-null    object 
 11  Primer contacto                 159 non-null    object 
 12  Estatus laboral                 121 

In [950]:
# //Drop columns no necesarias y/o con menos de un 40% de informacion

df_1 = df.drop(columns={"No.","Fecha de Nacimiento", "Edad","Razon de retiro", "Cargo","Feedback de los instructores"})

In [951]:
df_1 = df_1.rename(columns={'Precios':'Precios US'})

In [944]:
df_1.columns

Index(['Entrenamiento', 'Email', 'Egresado', 'Matricula', 'Precios US',
       'Ubicacion', 'Incluye pasantia', 'Metodo de pago', 'Primer contacto',
       'Estatus laboral', 'Condiciones de pago', 'Feedback de los Bootcamps',
       'Razon de eleccion del bootcamp', 'Proveedores de bootcamp',
       'Horario de los bootcamps'],
      dtype='object')

In [952]:
# Replace data - column Entrenamiento Eliminar redundancia de datos
for i in range(0, len(df_1['Entrenamiento'])):
      df_1['Entrenamiento'][i] = df_1['Entrenamiento'][i][12:]



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [953]:
# Replace data - column Incluye pasantia
dictionary = {"NO":0,"no":0,"No":0,"N":0, "Si":1, np.NaN:0, "?":0}  

df_1['Incluye pasantia'] = df_1['Incluye pasantia'].replace(dictionary)
df_1['Incluye pasantia'] = df_1['Incluye pasantia'] .astype(int)

In [957]:
# Replace data - column Precios

def limpiezaDeDatos(dataset,key,value):
   
    for i in range(0,len(dataset)):
        
        if type(dataset['Precios US'][i]) != float:
        
         dataset['Precios US'][i] = dataset['Precios US'][i].replace(key,value).strip()[0:5]
        else: 
            num = str(dataset['Precios US'][i])
            dataset['Precios US'][i] = num.replace('nan','0')
    return dataset

limpiezaDeDatos(df_1,'.','')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


,Entrenamiento,Email,Egresado,Matricula,Precios US,Ubicacion,Incluye pasantia,Metodo de pago,Primer contacto,Estatus laboral,Condiciones de pago,Feedback de los Bootcamps,Razon de eleccion del bootcamp,Proveedores de bootcamp,Horario de los bootcamps
0,Desarrollo Web con C#,holypally23@gmail.com,Egresado,20021,2900,"Luis F Thomen, Quisqueya, Santo Domingo",0,Transferencia,Instagram,Trabajando,Cuotas,Egresado,Crecimiento laboral / Networking / Aprender al...,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
1,Desarrollo Web con C#,greberyam@gmail.com,Retirado,20024,2900,C/respaldo Juan Luis Duquela 105 molinuevo enz...,0,Transferencia,Instagram,Trabajando,Cuotas,Retirado,Crecimiento laboral / Networking / Aprender al...,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
2,Desarrollo Web con C#,escleyxis.16@gmail.com,Egresado,20025,2900,"25 Este, Casa no, 34, Luperon Distrito Nacional",0,Transferencia,Instagram,Trabajando,Cuotas,Egresado,Crecimiento laboral / Networking / Aprender al...,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
3,Desarrollo Web con Java,pedtav23@gmail.com,Retirado,20028,0,NaN,0,NaN,NaN,NaN,Cuotas,Retirado,Crecimiento laboral / Networking / Aprender al...,Maestros de Talendig,"Martes, jueves y viernes de 6:00 pm a 8:00 pm"
4,Desarrollo Web con Java,scristianeulises@gmail.com,Egresado,20029,2900,"Armando Rodríguez , La Piedra, Boca Chica, San...",0,Transferencia,Instagram,Trabajando,Cuotas,Egresado,Crecimiento laboral / Networking / Aprender al...,Maestros de Talendig,"Martes, jueves y viernes de 6:00 pm a 8:00 pm"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Desarrollo Web,dylan.minaya12@gmail.com,Activo,20334,2900,"Cancino, Residencial Lomisa, Santo Domingo Este",0,Transferencia,Instagram,N/a,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Luner, miércoles y viernes de 8: pm a 10:00 pm"
156,Desarrollo Web,maria.liz.0733@gmail.com,Activo,20296,2900,"Edif H, Manzana 3, Santo Domingo Este",0,Transferencia,Instagram,N/a,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Luner, miércoles y viernes de 8: pm a 10:00 pm"
157,Data Science y Machine Learning en Python,estebanjim26@gmail.com,Activo,20335,2900,c/ jardines de los cerezos #32,1,Transferencia,WhatsApp,NaN,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Martes, Jueves y Sábados"
158,Data Science y Machine Learning en Python,elvis_mireles@hotmail.com,Activo,20328,2900,"Calle Sanchez #6, Barrio 30 de Mayo",1,Transferencia,WhatsApp,NaN,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Martes, Jueves y Sábados"


In [958]:
df_1.drop(151, inplace =True)

In [970]:
df_1[df_1['Precios US'] == 7] = 124

In [973]:
df_1[df_1['Precios US'] == 0.0] = 2900

In [959]:
df_1['Precios US'] = df_1['Precios US'].astype(float)

In [960]:
def fixLocation(location):
    
    cuidad = ['Este','Norte','Oeste']
    
    if type(location) == float or location == '?':
        location = 'Santo Domingo Este'
    
    for i in range(0, len(cuidad)):
       
        if cuidad[i] in location:
            location = 'Santo Domingo ' + cuidad[i]
      
    return location
    
df_1['Ubicacion'] = df_1['Ubicacion'].apply(lambda x : fixLocation(x))


In [961]:
# Cambiamos valores null
df_1['Metodo de pago'] = df_1['Metodo de pago'].apply(lambda x : 'Transferencia'  if (type(x) == float) else x) 

In [962]:
df_1['Primer contacto'] = df_1['Primer contacto'].apply(lambda x : 'Instagram'  if (type(x) == float) else x) 

In [963]:
df_1['Estatus laboral'] = df_1['Estatus laboral'].apply(lambda x : 0  if (type(x) == float) else 1) 

In [964]:
df_1['Condiciones de pago'] = df_1['Condiciones de pago'].apply(lambda x : 'Cuotas'  if (type(x) == float) else x) 

In [965]:
def fixEgresados(data):
    
    if data == 'Retirado':
        data = 0
    
    elif data == 'Egresado':
        data = 1
    else:
        data = 2
    return data

df_1['Egresado'] = df_1['Egresado'].apply(lambda x :fixEgresados(x))  

In [966]:
df_1['Matricula'] = df_1['Matricula'].astype(int)

In [967]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 159
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Entrenamiento                   159 non-null    object 
 1   Email                           159 non-null    object 
 2   Egresado                        159 non-null    int64  
 3   Matricula                       159 non-null    int64  
 4   Precios US                      159 non-null    float64
 5   Ubicacion                       159 non-null    object 
 6   Incluye pasantia                159 non-null    int64  
 7   Metodo de pago                  159 non-null    object 
 8   Primer contacto                 159 non-null    object 
 9   Estatus laboral                 159 non-null    int64  
 10  Condiciones de pago             159 non-null    object 
 11  Feedback de los Bootcamps       159 non-null    object 
 12  Razon de eleccion del bootcamp  159 

In [974]:
df_1['Precios US'].value_counts()

2900.0    122
124.0      13
3450.0     10
499.0       7
2990.0      5
299.0       2
Name: Precios US, dtype: int64

In [975]:
df_1['Razon de eleccion del bootcamp'] = df_1['Razon de eleccion del bootcamp'].apply(lambda x: "Aprender algo nuevo")
df_1['Razon de eleccion del bootcamp']

0      Aprender algo nuevo
1      Aprender algo nuevo
2      Aprender algo nuevo
3      Aprender algo nuevo
4      Aprender algo nuevo
              ...         
155    Aprender algo nuevo
156    Aprender algo nuevo
157    Aprender algo nuevo
158    Aprender algo nuevo
159    Aprender algo nuevo
Name: Razon de eleccion del bootcamp, Length: 159, dtype: object

In [976]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 159
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Entrenamiento                   159 non-null    object 
 1   Email                           159 non-null    object 
 2   Egresado                        159 non-null    int64  
 3   Matricula                       159 non-null    int64  
 4   Precios US                      159 non-null    float64
 5   Ubicacion                       159 non-null    object 
 6   Incluye pasantia                159 non-null    int64  
 7   Metodo de pago                  159 non-null    object 
 8   Primer contacto                 159 non-null    object 
 9   Estatus laboral                 159 non-null    int64  
 10  Condiciones de pago             159 non-null    object 
 11  Feedback de los Bootcamps       159 non-null    object 
 12  Razon de eleccion del bootcamp  159 

In [979]:
df_1.drop(3, inplace =True)

In [980]:
df_1

,Entrenamiento,Email,Egresado,Matricula,Precios US,Ubicacion,Incluye pasantia,Metodo de pago,Primer contacto,Estatus laboral,Condiciones de pago,Feedback de los Bootcamps,Razon de eleccion del bootcamp,Proveedores de bootcamp,Horario de los bootcamps
0,Desarrollo Web con C#,holypally23@gmail.com,1,20021,2900.0,"Luis F Thomen, Quisqueya, Santo Domingo",0,Transferencia,Instagram,1,Cuotas,Egresado,Aprender algo nuevo,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
1,Desarrollo Web con C#,greberyam@gmail.com,0,20024,2900.0,C/respaldo Juan Luis Duquela 105 molinuevo enz...,0,Transferencia,Instagram,1,Cuotas,Retirado,Aprender algo nuevo,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
2,Desarrollo Web con C#,escleyxis.16@gmail.com,1,20025,2900.0,Santo Domingo Este,0,Transferencia,Instagram,1,Cuotas,Egresado,Aprender algo nuevo,Maestros de Talendig,"Lunes, miércoles y viernes de 6:00pm a 8:00 pm"
4,Desarrollo Web con Java,scristianeulises@gmail.com,1,20029,2900.0,Santo Domingo Este,0,Transferencia,Instagram,1,Cuotas,Egresado,Aprender algo nuevo,Maestros de Talendig,"Martes, jueves y viernes de 6:00 pm a 8:00 pm"
5,Desarrollo Web con Java,125jeancarlos@gmail.com,1,20030,2900.0,Santo Domingo Norte,0,Transferencia,Instagram,1,Cuotas,Egresado,Aprender algo nuevo,Maestros de Talendig,"Martes, jueves y viernes de 6:00 pm a 8:00 pm"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Desarrollo Web,dylan.minaya12@gmail.com,2,20334,2900.0,Santo Domingo Este,0,Transferencia,Instagram,1,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Luner, miércoles y viernes de 8: pm a 10:00 pm"
156,Desarrollo Web,maria.liz.0733@gmail.com,2,20296,2900.0,Santo Domingo Este,0,Transferencia,Instagram,1,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Luner, miércoles y viernes de 8: pm a 10:00 pm"
157,Data Science y Machine Learning en Python,estebanjim26@gmail.com,2,20335,2900.0,c/ jardines de los cerezos #32,1,Transferencia,WhatsApp,0,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Martes, Jueves y Sábados"
158,Data Science y Machine Learning en Python,elvis_mireles@hotmail.com,2,20328,2900.0,"Calle Sanchez #6, Barrio 30 de Mayo",1,Transferencia,WhatsApp,0,Cuotas,Activo,Aprender algo nuevo,Maestros de Talendig,"Martes, Jueves y Sábados"
